In [ ]:
import pandas as pd
import difflib

In [ ]:
dfDicoBuch = pd.read_csv("1757_Buchanan-J.csv" , delimiter=";", encoding="utf8")
pronChar = pd.read_csv("Buchanan_PronChar_counts.csv", delimiter=";", encoding="utf8").dropna()

In [ ]:
#Transformation qui permet de rechercher la correspondance des lettres de Buchanan
def traduction(text):
    newWord = ""
    for eachLetter in text:
        if pronChar["char"].isin([eachLetter]).any():
            newWord += pronChar.loc[pronChar["char"] == eachLetter]["graph"].item()
    return newWord

Création de la colonne motVedette grâce à la fonction de traduction établie précédemment

In [ ]:
dfDicoBuch["motVedette"] = dfDicoBuch.apply(lambda x: traduction(x['pron']),1)

In [ ]:
dicoBigi = dfDicoBigi = pd.read_csv("2015_Bigi-B.dict", delimiter=" \[\] ", encoding="utf8", engine='python', names=['word','sampa'], skiprows=8 )
dfMappingSampaIpa = pd.read_csv("sampa-ipa.txt", delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])
dfMappingSampaIpa.drop_duplicates(subset=['sampa'], keep='first', inplace=True) 

Fonction de traduction du Sampa à l'IPA qui sera appelée pour tout le dictionnaire Bigi

In [ ]:
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""
    
    listValSampa = strSampa.split(" ")
    #print(listValSampa)
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['sampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

On ajoute la colonne IPA au dictionnaire Bigi

In [ ]:
dicoBigi['ipa'] = dicoBigi.apply(lambda x: sampa2ipa(x['sampa'], dfMappingSampaIpa),1)

In [ ]:
dfMerged = dfDicoBuch.merge(dicoBigi, left_on="motVedette", right_on="word", how="left")

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,word,sampa,ipa
1,2,ăbăʹft,NaN,NaN,NaN,p.,3.0,19,abaft,NaN,NaN,NaN
2,3,ăbaiʹſănce,F.,NaN,NaN,n.,4.0,19,abaisance,NaN,NaN,NaN
5,2,ăbāsʹe,F.,NaN,NaN,v.,7.0,19,abase,NaN,NaN,NaN
6,3,ăbāʹſ_e_ment,F.,NaN,NaN,n.,8.0,19,abasement,NaN,NaN,NaN
10,2,ăʹbbĕſs,NaN,NaN,NaN,n.,12.0,19,abbess,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15795,4,zōŏʹgrăphy,G.,NaN,NaN,n.,19.0,241,zoography,NaN,NaN,NaN
15796,4,zōŏʹphytès,G.,NaN,NaN,n.,20.0,241,zoophytes,NaN,NaN,NaN
15797,4,zōŏʹphŏric,G.,NaN,NaN,n.,21.0,241,zoophoric,NaN,NaN,NaN
15798,4,zōŏʹtŏmiſt,G.,NaN,NaN,n.,22.0,241,zootomist,NaN,NaN,NaN


In [ ]:
dfMerged.loc[pd.isna(dfMerged["ipa"])]

Un peu plus de 5000 mots n'ont pas été trouvés dans le dictionnaire Bigi. Il y a par exemple abaft ou abintestate qui semblent être des mots mal traduits dans le dictionnaire de Buchanan. On propose ainsi d'utiliser une bibliothèque qui permet de trouver l'occurence la plus proche. Exemple ci-dessous avec le mot abase.

In [ ]:
listWords = list(map(str, list(dicoBigi["word"])))
difflib.get_close_matches("abase", listWords, n=1)

['ambase']

On créé une nouvelle colonne : closest_word

In [ ]:
#La fonction get_close_matches retourne l'occurence la plus proche dans le bag of words du dictionnaire bigi
#Si le mot est trouvé, il est par conséquent l'occurence la plus proche
def closest_word(text):
    return difflib.get_close_matches(text, listWords, n=1)

In [ ]:
dfDicoBuch['closestWord'] = dfDicoBuch.apply(lambda x: closest_word(x['motVedette']),1)

Pour la suite de l'exo, on remplace la colonne motVedette par ces nouvelles valeurs puis on supprimer closestWord.

In [ ]:
dfDicoBuch["motVedette"] = df["closestWord"]
df.drop("closest_word")

On peut ainsi ré-effectuer le merge sur cette colonne

In [ ]:
dfMerged = dfDicoBuch.merge(dicoBigi, left_on="closestWord", right_on="word", how="left")

On enregistre ce dataframe

In [ ]:
dfMerged.to_csv("BuchananRich.csv", sep=";",encoding="utf8",index=False)

AttributeError: 'list' object has no attribute 'astype'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28a13d76-245b-4e36-8dd1-db1382e81cfa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>